In [1]:
!pip install gradio
!pip install gradio opencv-python-headless numpy matplotlib scikit-learn

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━

In [7]:
import gradio as gr
import cv2
import numpy as np
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import random

# Mock function to suggest dresses based on color palette
def suggest_dresses(colors):
    suggestions = []
    color_names = ['Red', 'Green', 'Blue', 'Yellow', 'Purple']
    patterns = ['Floral', 'Striped', 'Polka Dots', 'Plaid', 'Solid']
    image_urls = [
        "https://assets.myntassets.com/w_412,q_60,dpr_2,fl_progressive/assets/images/24880966/2023/9/12/20cea47a-68a6-4c4e-bc0e-f623e63d2c3f1694459541907DressBerryNavyBlueFloralPrintFitFlareMidiDress1.jpg",
        "https://assets.myntassets.com/w_412,q_60,dpr_2,fl_progressive/assets/images/6743809/2018/6/15/7efc6125-b0ce-4feb-b5fa-39feb16c304a1529064063573-RARE-Women-Navy-Blue-Solid-A-Line-Dress-9751529064063400-1.jpg",
        "https://assets.myntassets.com/w_412,q_60,dpr_2,fl_progressive/assets/images/23494484/2023/5/31/1ccf60dc-55a3-4083-bb01-17395a3838031685548904858WBluePrintFitFlareDress1.jpg",
        "https://assets.myntassets.com/w_412,q_60,dpr_2,fl_progressive/assets/images/28325090/2024/3/18/52702d6f-f36f-4378-8470-b366dd7549e31710727133197InWeavePolkaDotPrintA-LineDress1.jpg",
        "https://assets.myntassets.com/dpr_1.5,q_60,w_400,c_limit,fl_progressive/assets/images/29587666/2024/5/16/aea276e3-127f-4cae-b4e5-679da556d12b1715850527126STADOCheckedFitFlareMaxiDress1.jpg"
    ]

    random.shuffle(image_urls)  # Shuffle the list of image URLs

    for i, color in enumerate(colors):
        color_name = color_names[np.argmax(color)]
        pattern = np.random.choice(patterns)
        suggestion = {
            "description": f"Dress Color: {color_name}, Pattern: {pattern}",
            "image_url": image_urls[i % len(image_urls)]  # Select a different image URL for each suggestion
        }
        suggestions.append(suggestion)

    return suggestions

def analyze_image(image):
    # Convert the image to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect face
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    if len(faces) == 0:
        return "No face detected", None, None

    # Extract the face region
    (x, y, w, h) = faces[0]
    face_img = image[y:y+h, x:x+w]

    # Convert the face region to the right format for clustering
    face_img = face_img.reshape((face_img.shape[0] * face_img.shape[1], 3))

    # Use KMeans to find dominant colors
    kmeans = KMeans(n_clusters=5)
    kmeans.fit(face_img)

    colors = kmeans.cluster_centers_
    colors = colors.astype(int)

    # Create a palette image
    palette = np.zeros((50, 300, 3), np.uint8)
    step = 300 // 5
    for i in range(5):
        palette[:, i * step:(i + 1) * step, :] = colors[i]

    # Plot the palette
    plt.figure(figsize=(8, 2))
    plt.imshow(palette)
    plt.axis('off')
    plt.show()

    # Suggest dresses based on the colors
    suggestions = suggest_dresses(colors)

    # Prepare suggestions output
    suggestions_output = "\n".join([s['description'] for s in suggestions])
    images_output = [s['image_url'] for s in suggestions]

    return palette, suggestions_output, images_output

iface = gr.Interface(
    fn=analyze_image,
    inputs=gr.Image(type="numpy"),
    outputs=[
        gr.Image(type="numpy", label="Color Palette"),
        gr.Textbox(label="Dress Suggestions"),
        gr.Gallery(label="Dress Images")
    ],
    live=True
)

iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://35c2271cc3a7f9773e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
